# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [62]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.'''

## 2. Generate the mathematical model

In [72]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [73]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Facility Location and Customer Allocation

**Sets and Indices:**

*  $I$: Set of potential facility locations.
*  $J$: Set of customers.

**Parameters:**

*  $f_i$: Fixed annual activation cost of facility $i \in I$.
*  $c_{ij}$: Transportation cost of serving customer $j \in J$ from facility $i \in I$.
*  $d_j$: Annual demand of customer $j \in J$.
*  $C_i$: Maximum annual service volume of facility $i \in I$.

**Decision Variables:**

*  $x_{ij}$: Amount of customer $j$'s annual demand served by facility $i$.
*  $y_i$: Binary variable, equal to 1 if facility $i$ is opened, 0 otherwise. 

**Objective Function:**

Minimize the total cost, which includes facility activation costs and transportation costs:

 $$
 \text{Minimize} \sum_{i \in I} f_iy_i + \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} 
 $$

**Constraints:**

1. **Demand Satisfaction:** Each customer's demand must be met:
    $$
    \sum_{i \in I} x_{ij} = d_j,  \forall j \in J
    $$
2. **Facility Capacity:**  Service volume at each facility cannot exceed its capacity:
    $$
    \sum_{j \in J} x_{ij} \leq C_iy_i,  \forall i \in I
    $$
3. **Facility Activation:** Customers can only be served by opened facilities:
    $$
    x_{ij} \leq d_jy_i,  \forall i \in I,  \forall j \in J
    $$
4. **Binary Constraint:**
    $$
    y_i \in \{0, 1\},  \forall i \in I
    $$

This model is a mixed-integer linear programming (MILP) problem. Solving this model will provide the optimal facility locations to open ($y_i = 1$) and the optimal allocation of customers to facilities ($x_{ij}$ values) to minimize total cost. 


## 3. Generate the pyomo code

In [74]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [75]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with your actual data)
facilities = ['Facility1', 'Facility2', 'Facility3']
customers = ['Customer1', 'Customer2', 'Customer3', 'Customer4']

activation_cost = {'Facility1': 100, 'Facility2': 150, 'Facility3': 120}  
transportation_cost = {
    ('Facility1', 'Customer1'): 5,
    ('Facility1', 'Customer2'): 8,
    ('Facility1', 'Customer3'): 3,
    ('Facility1', 'Customer4'): 6,
    ('Facility2', 'Customer1'): 7,
    ('Facility2', 'Customer2'): 4,
    ('Facility2', 'Customer3'): 9,
    ('Facility2', 'Customer4'): 5,
    ('Facility3', 'Customer1'): 3,
    ('Facility3', 'Customer2'): 6,
    ('Facility3', 'Customer3'): 8,
    ('Facility3', 'Customer4'): 7,
}
demand = {'Customer1': 10, 'Customer2': 15, 'Customer3': 12, 'Customer4': 8}
capacity = {'Facility1': 30, 'Facility2': 40, 'Facility3': 25}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=facilities)
model.J = pyo.Set(initialize=customers)

# Parameters
model.f = pyo.Param(model.I, initialize=activation_cost)
model.c = pyo.Param(model.I, model.J, initialize=transportation_cost)
model.d = pyo.Param(model.J, initialize=demand)
model.C = pyo.Param(model.I, initialize=capacity)

# Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, domain=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.f[i]*model.y[i] for i in model.I) + sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_rule(model, j):
    return sum(model.x[i,j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_rule)

def capacity_rule(model, i):
    return sum(model.x[i,j] for j in model.J) <= model.C[i]*model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_rule)

def activation_rule(model, i, j):
    return model.x[i,j] <= model.d[j]*model.y[i]
model.activation_constraint = pyo.Constraint(model.I, model.J, rule=activation_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi', 'cplex' etc.
results = solver.solve(model)

# Print results
print(results)
print("Optimal Objective Value:", pyo.value(model.obj))

for i in model.I:
    if pyo.value(model.y[i]) == 1:
        print(f"Facility {i} is open.")
        for j in model.J:
            if pyo.value(model.x[i,j]) > 0:
                print(f"   Serving {pyo.value(model.x[i,j])} units of demand from customer {j}.") 
```

**Explanation:**

1. **Sample Data:** The code defines sample data for facilities, customers, costs, demand, and capacity. Make sure to replace these with your actual data.
2. **Model:** A Pyomo `ConcreteModel` is created to represent the optimization problem.
3. **Sets and Parameters:** Sets for facilities and customers, and parameters for costs, demand, and capacity are defined using `pyo.Set` and `pyo.Param`.
4. **Variables:** Decision variables `x` and `y` are defined using `pyo.Var`.
   -  `model.x` represents the amount of demand served.
   -  `model.y` represents whether a facility is open (1) or not (0).
5. **Objective Function:** The objective is to minimize total cost, which includes facility activation costs and transportation costs. This is defined using `pyo.Objective`.
6. **Constraints:** The constraints are:
   -  `demand_constraint`: Ensures all customer demand is met.
   -  `capacity_constraint`: Ensures that no facility exceeds its capacity.
   -  `activation_constraint`: Ensures customers are only served by open facilities.
7. **Solving:** A solver (in this case, 'glpk') is used to solve the model.
8. **Printing Results:** The code prints the solver status, objective value, open facilities, and the customer allocation to each open facility.

Remember to install the necessary packages (Pyomo and a solver like GLPK) before running this code. 


## 4. Input problem data and try running the generated code

In [85]:
import pyomo.environ as pyo

# Sample Data (Replace with your actual data)
facilities = ['Facility1', 'Facility2', 'Facility3']
customers = ['Customer1', 'Customer2', 'Customer3', 'Customer4', 'Customer5']

activation_cost = {'Facility1': 1000, 'Facility2': 1000, 'Facility3': 1000}  


transportation_cost = {
    ('Facility1', 'Customer1'): 4,
    ('Facility1', 'Customer2'): 5,
    ('Facility1', 'Customer3'): 6,
    ('Facility1', 'Customer4'): 8,
    ('Facility1', 'Customer5'): 10,
    ('Facility2', 'Customer1'): 6,
    ('Facility2', 'Customer2'): 4,
    ('Facility2', 'Customer3'): 3,
    ('Facility2', 'Customer4'): 5,
    ('Facility2', 'Customer5'): 8,
    ('Facility3', 'Customer1'): 9,
    ('Facility3', 'Customer2'): 7,
    ('Facility3', 'Customer3'): 4,
    ('Facility3', 'Customer4'): 3,
    ('Facility3', 'Customer5'): 4,
}
demand = {'Customer1': 80, 'Customer2': 270, 'Customer3': 250, 'Customer4': 160, 'Customer5': 180}
capacity = {'Facility1': 500, 'Facility2': 500, 'Facility3': 500}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=facilities)
model.J = pyo.Set(initialize=customers)

# Parameters
model.f = pyo.Param(model.I, initialize=activation_cost)
model.c = pyo.Param(model.I, model.J, initialize=transportation_cost)
model.d = pyo.Param(model.J, initialize=demand)
model.C = pyo.Param(model.I, initialize=capacity)

# Variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, domain=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.f[i]*model.y[i] for i in model.I) + sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_rule(model, j):
    return sum(model.x[i,j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_rule)

def capacity_rule(model, i):
    return sum(model.x[i,j] for j in model.J) <= model.C[i]*model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_rule)

def activation_rule(model, i, j):
    return model.x[i,j] <= model.d[j]*model.y[i]
model.activation_constraint = pyo.Constraint(model.I, model.J, rule=activation_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi', 'cplex' etc.
results = solver.solve(model)

# Print results
print(results)
print("Optimal Objective Value:", pyo.value(model.obj))

for i in model.I:
    if pyo.value(model.y[i]) == 1:
        print(f"Facility {i} is open.")
        for j in model.J:
            if pyo.value(model.x[i,j]) > 0:
                print(f"   Serving {pyo.value(model.x[i,j])} units of demand from customer {j}.") 


Problem: 
- Name: unknown
  Lower bound: 5610.0
  Upper bound: 5610.0
  Number of objectives: 1
  Number of constraints: 23
  Number of variables: 18
  Number of nonzeros: 63
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04091644287109375
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Objective Value: 5610.0
Facility Facility2 is open.
   Serving 80.0 units of demand from customer Customer1.
   Serving 270.0 units of demand from customer Customer2.
   Serving 150.0 units of demand from customer Customer3.
Facility Facility3 is open.
   Serving 100.0 units of demand from customer Customer3.
   Serving 160.0 units of demand from customer Customer4.
   Serving 180.0 units of demand from customer Customer5.


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [86]:
print(response.text)

## Mathematical Optimization Model for Facility Location and Customer Allocation

**Sets and Indices:**

*  $I$: Set of potential facility locations.
*  $J$: Set of customers.

**Parameters:**

*  $f_i$: Fixed annual activation cost of facility $i \in I$.
*  $c_{ij}$: Transportation cost of serving customer $j \in J$ from facility $i \in I$.
*  $d_j$: Annual demand of customer $j \in J$.
*  $C_i$: Maximum annual service volume of facility $i \in I$.

**Decision Variables:**

*  $x_{ij}$: Amount of customer $j$'s annual demand served by facility $i$.
*  $y_i$: Binary variable, equal to 1 if facility $i$ is opened, 0 otherwise. 

**Objective Function:**

Minimize the total cost, which includes facility activation costs and transportation costs:

 $$
 \text{Minimize} \sum_{i \in I} f_iy_i + \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} 
 $$

**Constraints:**

1. **Demand Satisfaction:** Each customer's demand must be met:
    $$
    \sum_{i \in I} x_{ij} = d_j,  \forall j \in J
    $$
2. **F

In [87]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with your actual data)
facilities = ['Facility1', 'Facility2', 'Facility3']
customers = ['Customer1', 'Customer2', 'Customer3', 'Customer4']

activation_cost = {'Facility1': 100, 'Facility2': 150, 'Facility3': 120}  
transportation_cost = {
    ('Facility1', 'Customer1'): 5,
    ('Facility1', 'Customer2'): 8,
    ('Facility1', 'Customer3'): 3,
    ('Facility1', 'Customer4'): 6,
    ('Facility2', 'Customer1'): 7,
    ('Facility2', 'Customer2'): 4,
    ('Facility2', 'Customer3'): 9,
    ('Facility2', 'Customer4'): 5,
    ('Facility3', 'Customer1'): 3,
    ('Facility3', 'Customer2'): 6,
    ('Facility3', 'Customer3'): 8,
    ('Facility3', 'Customer4'): 7,
}
demand = {'Customer1': 10, 'Customer2': 15, 'Customer3': 12, 'Customer4': 8}
capacity = {'Facility1': 30, 'Facility2': 40, 'Facility3': 25}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=facilities)
model.J = pyo.Set(initialize=customers)

# Parame